In [ ]:
# Configuration - All settings at top of notebook
print("🏆 INFORMATION EXTRACTION COMPARISON: Llama 3.2 Vision vs InternVL3")
print("🎯 Focus: Information extraction performance with structured YAML prompts")
print("=" * 80)

# CONFIGURATION - All settings defined here
CONFIG = {
    "model_paths": {
        "llama": "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision",
        "internvl": "/home/jovyan/nfs_share/models/InternVL3-8B"
    },
    "extraction_prompt": """<|image|>Extract key information in YAML format:

store_name: ""
date: ""
total: ""

Output only YAML. Stop after completion.""",
    "max_new_tokens": 64,
    "enable_quantization": True,
    "test_models": ["llama", "internvl"],
    "test_images": [
        ("image14.png", "TAX_INVOICE"),
        ("image65.png", "TAX_INVOICE"), 
        ("image71.png", "TAX_INVOICE"),
        ("image74.png", "TAX_INVOICE"),
        ("image205.png", "FUEL_RECEIPT"),
        ("image23.png", "TAX_INVOICE"),
        ("image45.png", "TAX_INVOICE"),
        ("image1.png", "BANK_STATEMENT"),
        ("image203.png", "BANK_STATEMENT"),
        ("image204.png", "FUEL_RECEIPT"),
        ("image206.png", "OTHER"),
    ]
}

print(f"✅ Configuration loaded:")
print(f"   - Models: {', '.join(CONFIG['test_models'])}")
print(f"   - Documents: {len(CONFIG['test_images'])} test images")
print(f"   - Format: Structured YAML prompts")
print(f"   - Max tokens: {CONFIG['max_new_tokens']}")
print(f"   - Quantization: {CONFIG['enable_quantization']}")
print(f"\n📋 Ready for step-by-step information extraction comparison")

# Imports and Modular Classes
import time
import torch
import json
import re
import gc
from pathlib import Path
from PIL import Image
from typing import Dict, List, Tuple, Optional, Any

class MemoryManager:
    """Memory management utilities for model testing"""
    
    @staticmethod
    def cleanup_gpu_memory():
        """Minimize memory footprint as requested"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    @staticmethod
    def get_memory_usage() -> Dict[str, float]:
        """Get current GPU memory usage in GB"""
        if torch.cuda.is_available():
            return {
                "allocated": torch.cuda.memory_allocated() / 1024**3,
                "reserved": torch.cuda.memory_reserved() / 1024**3
            }
        return {"allocated": 0.0, "reserved": 0.0}

class UltraAggressiveRepetitionController:
    """Business document repetition detection and cleanup"""
    
    def __init__(self, word_threshold: float = 0.15, phrase_threshold: int = 2):
        self.word_threshold = word_threshold
        self.phrase_threshold = phrase_threshold
        
        # Business document specific repetition patterns
        self.toxic_patterns = [
            r"THANK YOU FOR SHOPPING WITH US[^.]*",
            r"All prices include GST where applicable[^.]*",
            r"applicable\.\s*applicable\.",
            r"GST where applicable[^.]*applicable",
            r"\\+[a-zA-Z]*\{[^}]*\}",  # LaTeX artifacts
            r"\(\s*\)",  # Empty parentheses
            r"[.-]\s*THANK YOU",
        ]
    
    def clean_response(self, response: str) -> str:
        """Clean business document extraction response"""
        if not response or len(response.strip()) == 0:
            return ""
        
        # Remove toxic business document patterns
        response = self._remove_business_patterns(response)
        
        # Remove repetitive words and phrases
        response = self._remove_word_repetition(response)
        response = self._remove_phrase_repetition(response)
        
        # Clean artifacts
        response = re.sub(r'\s+', ' ', response)
        response = re.sub(r'[.]{2,}', '.', response)
        response = re.sub(r'[!]{2,}', '!', response)
        
        return response.strip()
    
    def _remove_business_patterns(self, text: str) -> str:
        """Remove business document specific repetitive patterns"""
        for pattern in self.toxic_patterns:
            text = re.sub(pattern, "", text, flags=re.IGNORECASE)
        
        # Remove excessive "applicable" repetition
        text = re.sub(r'(applicable\.\s*){2,}', 'applicable. ', text, flags=re.IGNORECASE)
        
        return text
    
    def _remove_word_repetition(self, text: str) -> str:
        """Remove word repetition in business documents"""
        # Remove consecutive identical words
        text = re.sub(r'\b(\w+)(\s+\1){1,}', r'\1', text, flags=re.IGNORECASE)
        
        return text
    
    def _remove_phrase_repetition(self, text: str) -> str:
        """Remove phrase repetition"""
        for phrase_length in range(2, 7):
            pattern = r'\b((?:\w+\s+){' + str(phrase_length-1) + r'}\w+)(\s+\1){1,}'
            text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE)
        
        return text

class YAMLExtractionAnalyzer:
    """Analyzer for YAML extraction results"""
    
    @staticmethod
    def analyze(response: str, img_name: str) -> Dict[str, Any]:
        """Analyze YAML extraction results with consistent format"""
        response_clean = response.strip()
        
        # Detect YAML format
        is_yaml = bool(re.search(r'(store_name:|date:|total:)', response_clean, re.IGNORECASE))
        
        # Extract data from YAML or text
        store_match = re.search(r'store_name:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
        date_match = re.search(r'date:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)  
        total_match = re.search(r'total:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
        
        # Fallback detection for non-YAML responses
        if not store_match:
            store_match = re.search(r'(spotlight|store)', response_clean, re.IGNORECASE)
        if not date_match:
            date_match = re.search(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', response_clean)
        if not total_match:
            total_match = re.search(r'(\$\d+\.\d{2}|\$\d+)', response_clean)
        
        has_store = bool(store_match)
        has_date = bool(date_match)
        has_total = bool(total_match)
        
        extraction_score = sum([has_store, has_date, has_total])
        
        return {
            "img_name": img_name,
            "response": response_clean,
            "is_yaml": is_yaml,
            "has_store": has_store,
            "has_date": has_date,
            "has_total": has_total,
            "extraction_score": extraction_score,
            "successful": extraction_score >= 2  # At least 2/3 fields
        }

class DatasetManager:
    """Dataset verification and management"""
    
    def __init__(self, datasets_path: str = "datasets"):
        self.datasets_path = Path(datasets_path)
    
    def verify_images(self, test_images: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """Verify that test images exist and return verified list"""
        verified_images = []
        
        for img_name, doc_type in test_images:
            img_path = self.datasets_path / img_name
            if img_path.exists():
                verified_images.append((img_name, doc_type))
        
        return verified_images
    
    def print_verification_report(self, test_images: List[Tuple[str, str]], verified_images: List[Tuple[str, str]]):
        """Print dataset verification report"""
        print("📊 DATASET VERIFICATION")
        print("=" * 50)
        
        for img_name, doc_type in test_images:
            img_path = self.datasets_path / img_name
            if img_path.exists():
                print(f"   ✅ {img_name:<12} → {doc_type}")
            else:
                print(f"   ❌ {img_name:<12} → {doc_type} (MISSING)")
        
        print(f"\n📋 Dataset Summary:")
        print(f"   - Expected: {len(test_images)} documents")
        print(f"   - Found: {len(verified_images)} documents")
        print(f"   - Missing: {len(test_images) - len(verified_images)} documents")
        
        if len(verified_images) == 0:
            print("❌ No test images found! Check datasets/ directory")
            raise FileNotFoundError("No test images found")
        elif len(verified_images) < len(test_images):
            print("⚠️ Some test images missing but proceeding with available images")
        else:
            print("✅ All test images found")

# Initialize global utilities
memory_manager = MemoryManager()
repetition_controller = UltraAggressiveRepetitionController()
yaml_analyzer = YAMLExtractionAnalyzer()
dataset_manager = DatasetManager()

print("✅ Modular classes initialized:")
print("   - MemoryManager for GPU cleanup")
print("   - UltraAggressiveRepetitionController for text cleanup")
print("   - YAMLExtractionAnalyzer for results analysis")
print("   - DatasetManager for image verification")

In [ ]:
# Dataset Verification
# Use the modular DatasetManager class

verified_extraction_images = dataset_manager.verify_images(CONFIG["test_images"])
dataset_manager.print_verification_report(CONFIG["test_images"], verified_extraction_images)

print(f"\n🔬 YAML Extraction Prompt:")
print(f"   {CONFIG['extraction_prompt'][:60]}...")
print(f"\n📋 Ready for model testing")

In [ ]:
# Model Loading Classes
class LlamaModelLoader:
    """Modular Llama model loader with validation"""
    
    @staticmethod
    def load_model(model_path: str, enable_quantization: bool = True):
        """Load Llama model with proper configuration"""
        from transformers import AutoProcessor, MllamaForConditionalGeneration
        from transformers import BitsAndBytesConfig
        
        processor = AutoProcessor.from_pretrained(
            model_path, trust_remote_code=True, local_files_only=True
        )
        
        model_kwargs = {
            "torch_dtype": torch.float16,
            "local_files_only": True
        }
        
        if enable_quantization:
            quantization_config = BitsAndBytesConfig(
                load_in_8bit=True,
                llm_int8_enable_fp32_cpu_offload=True,
                llm_int8_skip_modules=["vision_tower", "multi_modal_projector"],
            )
            model_kwargs["quantization_config"] = quantization_config
        
        model = MllamaForConditionalGeneration.from_pretrained(
            model_path, **model_kwargs
        ).eval()
        
        return model, processor
    
    @staticmethod
    def run_inference(model, processor, prompt: str, image, max_new_tokens: int = 64):
        """Run inference with proper device handling"""
        inputs = processor(text=prompt, images=image, return_tensors="pt")
        device = next(model.parameters()).device
        if device.type != "cpu":
            device_target = str(device).split(":")[0]
            inputs = {k: v.to(device_target) if hasattr(v, "to") else v for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=processor.tokenizer.eos_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                use_cache=True,
            )
        
        raw_response = processor.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        
        # Cleanup tensors immediately
        del inputs, outputs
        
        return raw_response

class InternVLModelLoader:
    """Modular InternVL model loader with validation"""
    
    @staticmethod
    def load_model(model_path: str, enable_quantization: bool = True):
        """Load InternVL model with proper configuration"""
        from transformers import AutoModel, AutoTokenizer
        
        tokenizer = AutoTokenizer.from_pretrained(
            model_path, trust_remote_code=True, local_files_only=True
        )
        
        model_kwargs = {
            "trust_remote_code": True,
            "torch_dtype": torch.bfloat16,
            "local_files_only": True
        }
        
        if enable_quantization:
            model_kwargs["load_in_8bit"] = True
        
        model = AutoModel.from_pretrained(
            model_path, **model_kwargs
        ).eval()
        
        return model, tokenizer
    
    @staticmethod
    def run_inference(model, tokenizer, prompt: str, image, max_new_tokens: int = 64):
        """Run inference with proper image preprocessing"""
        import torchvision.transforms as T
        from torchvision.transforms.functional import InterpolationMode
        
        transform = T.Compose([
            T.Resize((448, 448), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        
        pixel_values = transform(image).unsqueeze(0)
        if torch.cuda.is_available():
            pixel_values = pixel_values.cuda().to(torch.bfloat16).contiguous()
        
        raw_response = model.chat(
            tokenizer=tokenizer,
            pixel_values=pixel_values,
            question=prompt,
            generation_config={"max_new_tokens": max_new_tokens, "do_sample": False}
        )
        
        if isinstance(raw_response, tuple):
            raw_response = raw_response[0]
        
        # Cleanup tensors immediately
        del pixel_values
        
        return raw_response

def validate_model(model_loader_class, model_path: str, config: Dict) -> Tuple[bool, Optional[Any], Optional[Any], float]:
    """Generic model validation function"""
    memory_manager.cleanup_gpu_memory()
    model_start_time = time.time()
    
    try:
        print(f"Loading model from {model_path}...")
        
        model, processor_or_tokenizer = model_loader_class.load_model(
            model_path, config["enable_quantization"]
        )
        
        model_load_time = time.time() - model_start_time
        print(f"✅ Model loaded in {model_load_time:.1f}s")
        
        # Test image14.png first to validate
        print(f"\n🔍 Testing image14.png for validation...")
        img_path = dataset_manager.datasets_path / "image14.png"
        image = Image.open(img_path).convert("RGB")
        
        inference_start = time.time()
        
        raw_response = model_loader_class.run_inference(
            model, processor_or_tokenizer, config["extraction_prompt"], 
            image, config["max_new_tokens"]
        )
        
        inference_time = time.time() - inference_start
        cleaned_response = repetition_controller.clean_response(raw_response)
        validation_analysis = yaml_analyzer.analyze(cleaned_response, "image14.png")
        
        print(f"   Validation result: {validation_analysis['extraction_score']}/3 fields extracted")
        print(f"   YAML format: {'✅' if validation_analysis['is_yaml'] else '❌'}")
        print(f"   Inference time: {inference_time:.1f}s")
        
        if validation_analysis["successful"]:
            print(f"✅ Validation passed - model working")
            return True, model, processor_or_tokenizer, model_load_time
        else:
            print(f"❌ Validation failed - check prompts and model")
            del model, processor_or_tokenizer
            memory_manager.cleanup_gpu_memory()
            return False, None, None, model_load_time
            
    except Exception as e:
        print(f"❌ Model failed to load: {str(e)[:100]}...")
        memory_manager.cleanup_gpu_memory()
        return False, None, None, 0.0

print("✅ Model loader classes defined:")
print("   - LlamaModelLoader with validation")
print("   - InternVLModelLoader with validation")
print("   - validate_model() generic function")

In [ ]:
# Test Llama Model
print("🔬 TESTING LLAMA MODEL")
print("=" * 50)

# Initialize results storage
extraction_results = {
    "llama": {"documents": [], "successful": 0, "total_time": 0},
    "internvl": {"documents": [], "successful": 0, "total_time": 0}
}

# Use modular validation function
llama_valid, llama_model, llama_processor, llama_load_time = validate_model(
    LlamaModelLoader, 
    CONFIG["model_paths"]["llama"], 
    CONFIG
)

if llama_valid:
    print("✅ Llama model ready for full testing")
    # Store for next cell
    globals()['llama_model'] = llama_model
    globals()['llama_processor'] = llama_processor
    globals()['llama_load_time'] = llama_load_time
else:
    print("❌ Llama model validation failed")

In [ ]:
# Test InternVL Model  
print("🔬 TESTING INTERNVL MODEL")
print("=" * 50)

# Use modular validation function
internvl_valid, internvl_model, internvl_tokenizer, internvl_load_time = validate_model(
    InternVLModelLoader,
    CONFIG["model_paths"]["internvl"],
    CONFIG
)

if internvl_valid:
    print("✅ InternVL model ready for full testing")
    # Store for next cell
    globals()['internvl_model'] = internvl_model
    globals()['internvl_tokenizer'] = internvl_tokenizer
    globals()['internvl_load_time'] = internvl_load_time
else:
    print("❌ InternVL model validation failed")

In [ ]:
# Run Full Extraction Test - Llama
if 'llama_model' in globals() and llama_model is not None:
    print("🔍 FULL EXTRACTION TEST - LLAMA")
    print("=" * 50)
    
    total_inference_time = 0
    
    for i, (img_name, doc_type) in enumerate(verified_extraction_images, 1):
        try:
            img_path = dataset_manager.datasets_path / img_name
            image = Image.open(img_path).convert("RGB")
            
            inference_start = time.time()
            
            raw_response = LlamaModelLoader.run_inference(
                llama_model, llama_processor, CONFIG["extraction_prompt"],
                image, CONFIG["max_new_tokens"]
            )
            
            inference_time = time.time() - inference_start
            total_inference_time += inference_time
            
            cleaned_response = repetition_controller.clean_response(raw_response)
            analysis = yaml_analyzer.analyze(cleaned_response, img_name)
            analysis["inference_time"] = inference_time
            analysis["doc_type"] = doc_type
            
            extraction_results["llama"]["documents"].append(analysis)
            
            if analysis["successful"]:
                extraction_results["llama"]["successful"] += 1
            
            # Consistent output format as requested
            status = "✅" if analysis["successful"] else "❌"
            yaml_status = "Y" if analysis["is_yaml"] else "T"
            print(f"   {i:2d}. {img_name:<12} {status} {inference_time:.1f}s | {yaml_status} | {analysis['extraction_score']}/3")
            
            # Immediate tensor cleanup - minimizing memory footprint
            del image
            
            # Periodic GPU cleanup every 3 images
            if i % 3 == 0:
                memory_manager.cleanup_gpu_memory()
            
        except Exception as e:
            print(f"   {i:2d}. {img_name:<12} ❌ Error: {str(e)[:30]}...")
    
    extraction_results["llama"]["total_time"] = total_inference_time
    extraction_results["llama"]["avg_time"] = total_inference_time / len(verified_extraction_images)
    
    print(f"\n📊 Llama Results:")
    print(f"   Success rate: {extraction_results['llama']['successful']}/{len(verified_extraction_images)}")
    print(f"   Average time: {extraction_results['llama']['avg_time']:.1f}s per document")
    
    # Cleanup Llama model to free memory for InternVL
    del llama_model, llama_processor
    memory_manager.cleanup_gpu_memory()
    
else:
    print("⚠️ Llama model not available - skipping full test")

In [ ]:
# Run Full Extraction Test - InternVL
if 'internvl_model' in globals() and internvl_model is not None:
    print("🔍 FULL EXTRACTION TEST - INTERNVL")
    print("=" * 50)
    
    total_inference_time = 0
    
    for i, (img_name, doc_type) in enumerate(verified_extraction_images, 1):
        try:
            img_path = dataset_manager.datasets_path / img_name
            image = Image.open(img_path).convert("RGB")
            
            inference_start = time.time()
            
            raw_response = InternVLModelLoader.run_inference(
                internvl_model, internvl_tokenizer, CONFIG["extraction_prompt"],
                image, CONFIG["max_new_tokens"]
            )
            
            inference_time = time.time() - inference_start
            total_inference_time += inference_time
            
            cleaned_response = repetition_controller.clean_response(raw_response)
            analysis = yaml_analyzer.analyze(cleaned_response, img_name)
            analysis["inference_time"] = inference_time
            analysis["doc_type"] = doc_type
            
            extraction_results["internvl"]["documents"].append(analysis)
            
            if analysis["successful"]:
                extraction_results["internvl"]["successful"] += 1
            
            # Consistent output format as requested
            status = "✅" if analysis["successful"] else "❌"
            yaml_status = "Y" if analysis["is_yaml"] else "T"
            print(f"   {i:2d}. {img_name:<12} {status} {inference_time:.1f}s | {yaml_status} | {analysis['extraction_score']}/3")
            
            # Immediate tensor cleanup - minimizing memory footprint
            del image
            
            # Periodic GPU cleanup every 3 images
            if i % 3 == 0:
                memory_manager.cleanup_gpu_memory()
            
        except Exception as e:
            print(f"   {i:2d}. {img_name:<12} ❌ Error: {str(e)[:30]}...")
    
    extraction_results["internvl"]["total_time"] = total_inference_time
    extraction_results["internvl"]["avg_time"] = total_inference_time / len(verified_extraction_images)
    
    print(f"\n📊 InternVL Results:")
    print(f"   Success rate: {extraction_results['internvl']['successful']}/{len(verified_extraction_images)}")
    print(f"   Average time: {extraction_results['internvl']['avg_time']:.1f}s per document")
    
    # Cleanup InternVL model 
    del internvl_model, internvl_tokenizer
    memory_manager.cleanup_gpu_memory()
    
else:
    print("⚠️ InternVL model not available - skipping full test")

In [ ]:
# Final Comparison and Recommendation
class ResultsAnalyzer:
    """Modular results analysis and comparison"""
    
    @staticmethod
    def print_final_comparison(extraction_results: Dict, verified_images: List):
        """Print final comparison between models"""
        print(f"\n{'=' * 80}")
        print("🏆 FINAL RECOMMENDATION: BEST MODEL FOR INFORMATION EXTRACTION")
        print(f"{'=' * 80}")
        
        # Compare both models' performance
        llama_success = 0
        llama_total = 0
        llama_avg_time = 0
        internvl_success = 0
        internvl_total = 0
        internvl_avg_time = 0
        
        if extraction_results["llama"]["documents"]:
            llama_total = len(extraction_results["llama"]["documents"])
            llama_success = extraction_results["llama"]["successful"]
            llama_avg_time = extraction_results["llama"]["avg_time"]
        
        if extraction_results["internvl"]["documents"]:
            internvl_total = len(extraction_results["internvl"]["documents"])
            internvl_success = extraction_results["internvl"]["successful"]
            internvl_avg_time = extraction_results["internvl"]["avg_time"]
        
        print(f"📊 INFORMATION EXTRACTION COMPARISON:")
        print(f"{'Model':<12} {'Success Rate':<15} {'Avg Time':<12} {'Best For'}")
        print("-" * 60)
        
        if llama_total > 0:
            llama_rate = llama_success / llama_total * 100
            print(f"{'LLAMA':<12} {llama_rate:.1f}% ({llama_success}/{llama_total}){'':<5} {llama_avg_time:.1f}s{'':<7} Large context")
        
        if internvl_total > 0:
            internvl_rate = internvl_success / internvl_total * 100
            print(f"{'INTERNVL':<12} {internvl_rate:.1f}% ({internvl_success}/{internvl_total}){'':<5} {internvl_avg_time:.1f}s{'':<7} Production speed")
        
        # Make recommendation
        if internvl_total > 0 and llama_total > 0:
            internvl_rate = internvl_success / internvl_total * 100
            llama_rate = llama_success / llama_total * 100
            
            if internvl_rate > llama_rate:
                recommended = "INTERNVL"
                reason = f"Higher success rate ({internvl_rate:.1f}% vs {llama_rate:.1f}%) and faster inference"
            elif llama_rate > internvl_rate:
                recommended = "LLAMA"
                reason = f"Higher success rate ({llama_rate:.1f}% vs {internvl_rate:.1f}%)"
            else:
                recommended = "INTERNVL"
                reason = f"Equal success rate but {internvl_avg_time/llama_avg_time:.1f}x faster inference"
            
            print(f"\n🥇 RECOMMENDED FOR INFORMATION EXTRACTION: {recommended}")
            print(f"   Reason: {reason}")
            print(f"   Use case: Business document processing (receipts, invoices, statements)")
        elif internvl_total > 0:
            print(f"\n🥇 RECOMMENDED: INTERNVL (only model tested successfully)")
        elif llama_total > 0:
            print(f"\n🥇 RECOMMENDED: LLAMA (only model tested successfully)")
        else:
            print(f"\n⚠️ No successful tests - investigate model loading issues")
        
        print(f"\n✅ COMPLETE: Information extraction performance comparison finished!")
        print(f"📋 This answers the user's question about best model for their information extraction job")

# Use the modular analyzer
results_analyzer = ResultsAnalyzer()
results_analyzer.print_final_comparison(extraction_results, verified_extraction_images)

# Show the YAML prompt being used
print(f"\n🔬 YAML PROMPT USED:")
print(f"{'='*50}")
print(CONFIG["extraction_prompt"])
print(f"{'='*50}")
print(f"✅ Confirmed: Using structured YAML prompts (NOT JSON)")

In [10]:
# Multi-Document Classification - Improved Llama 3.2 Vision Prompting
print("🏛️ COMPREHENSIVE TAXPAYER DOCUMENT CLASSIFICATION TEST")
print("🧪 Using IMPROVED research-based prompting techniques")
print("=" * 80)

import time
import torch
import gc
import json
from pathlib import Path
from PIL import Image
from collections import defaultdict
from transformers import AutoProcessor, MllamaForConditionalGeneration
from transformers import AutoModel, AutoTokenizer
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# Memory management function
def cleanup_gpu_memory():
    """Aggressive GPU memory cleanup"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        memory_allocated = torch.cuda.memory_allocated() / 1024**3
        memory_reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"   GPU Memory: {memory_allocated:.2f}GB allocated, {memory_reserved:.2f}GB reserved")

# Standard document types
DOCUMENT_TYPES = [
    "FUEL_RECEIPT", "BUSINESS_RECEIPT", "TAX_INVOICE", "BANK_STATEMENT",
    "MEAL_RECEIPT", "ACCOMMODATION_RECEIPT", "TRAVEL_DOCUMENT", 
    "PARKING_TOLL_RECEIPT", "PROFESSIONAL_SERVICES", "EQUIPMENT_SUPPLIES", "OTHER"
]

# Human annotated ground truth
test_images_with_annotations = [
    ("image14.png", "TAX_INVOICE"),
    ("image65.png", "TAX_INVOICE"),
    ("image71.png", "TAX_INVOICE"),
    ("image74.png", "TAX_INVOICE"),
    ("image205.png", "FUEL_RECEIPT"),
    ("image23.png", "TAX_INVOICE"),
    ("image45.png", "TAX_INVOICE"),
    ("image1.png", "BANK_STATEMENT"),
    ("image203.png", "BANK_STATEMENT"),
    ("image204.png", "FUEL_RECEIPT"),
    ("image206.png", "OTHER"),
]

# Verify test images exist
datasets_path = Path("datasets")
verified_test_images = []
verified_ground_truth = {}

for img_name, annotation in test_images_with_annotations:
    img_path = datasets_path / img_name
    if img_path.exists():
        verified_test_images.append(img_name)
        verified_ground_truth[img_name] = annotation
    else:
        print(f"⚠️ Missing: {img_name} (expected: {annotation})")

print(f"📊 Testing {len(verified_test_images)} documents with HUMAN ANNOTATIONS:")
for i, img_name in enumerate(verified_test_images, 1):
    annotation = verified_ground_truth[img_name]
    print(f"   {i}. {img_name:<12} → {annotation}")

# IMPROVED classification prompts based on research
classification_prompts = {
    "json_format": f"""<|image|>Classify this business document in JSON format:
{{
  "document_type": ""
}}

Categories: {', '.join(DOCUMENT_TYPES)}
Return only valid JSON, no explanations.""",
    
    "simple_format": f"""<|image|>What type of business document is this?

Choose from: {', '.join(DOCUMENT_TYPES)}

Answer with one category only:""",
    
    "ultra_simple": "<|image|>Document type:",
}

print(f"\n🧪 Available classification prompts:")
for name, prompt in classification_prompts.items():
    print(f"   - {name}: {len(prompt)} chars")

# Results storage with accuracy tracking
multi_doc_results = {
    "llama": {"classifications": [], "times": [], "errors": [], "correct": 0, "total": 0},
    "internvl": {"classifications": [], "times": [], "errors": [], "correct": 0, "total": 0}
}

# Test both models with IMPROVED prompting
for model_name in ["llama", "internvl"]:
    print(f"\n{'=' * 60}")
    print(f"🔍 TESTING {model_name.upper()} WITH IMPROVED PROMPTING")
    print(f"{'=' * 60}")
    
    # AGGRESSIVE pre-cleanup before loading model
    print(f"🧹 Pre-cleanup for {model_name}...")
    for var in ['model', 'processor', 'tokenizer', 'inputs', 'outputs', 'pixel_values']:
        if var in locals():
            del locals()[var]
        if var in globals():
            del globals()[var]
    cleanup_gpu_memory()
    
    model_start_time = time.time()
    
    # Select best prompt for model type
    if model_name == "llama":
        # Use simple format to avoid safety triggers
        classification_prompt = classification_prompts["simple_format"]
        print(f"📝 Using SIMPLE FORMAT prompt (research-based)")
    else:
        # InternVL can handle JSON better
        classification_prompt = classification_prompts["json_format"]
        print(f"📝 Using JSON FORMAT prompt")
    
    try:
        # Load model using ROBUST patterns from cell 3
        model_path = CONFIG["model_paths"][model_name]
        print(f"Loading {model_name} model from {model_path}...")
        
        if model_name == "llama":
            print(f"🔄 Loading Llama (will use ~6-8GB GPU memory)...")
            
            processor = AutoProcessor.from_pretrained(
                model_path, trust_remote_code=True, local_files_only=True
            )
            
            model_loading_args = {
                "low_cpu_mem_usage": True,
                "torch_dtype": torch.float16,
                "device_map": "cuda:0" if torch.cuda.is_available() else "cpu",
                "local_files_only": True
            }
            
            if CONFIG["enable_quantization"] and torch.cuda.is_available():
                try:
                    from transformers import BitsAndBytesConfig
                    quantization_config = BitsAndBytesConfig(
                        load_in_8bit=True,
                        llm_int8_enable_fp32_cpu_offload=True,
                        llm_int8_skip_modules=["vision_tower", "multi_modal_projector"],
                    )
                    model_loading_args["quantization_config"] = quantization_config
                    print("✅ Using 8-bit quantization")
                except ImportError:
                    pass
            
            model = MllamaForConditionalGeneration.from_pretrained(
                model_path, **model_loading_args
            ).eval()
            
        elif model_name == "internvl":
            print(f"🔄 Loading InternVL (will use ~4-6GB GPU memory)...")
            
            tokenizer = AutoTokenizer.from_pretrained(
                model_path, trust_remote_code=True, local_files_only=True
            )
            
            model_kwargs = {
                "low_cpu_mem_usage": True,
                "trust_remote_code": True,
                "torch_dtype": torch.bfloat16,
                "local_files_only": True
            }
            
            if CONFIG["enable_quantization"] and torch.cuda.is_available():
                try:
                    model_kwargs["load_in_8bit"] = True
                    print("✅ 8-bit quantization enabled")
                except Exception:
                    pass
            
            model = AutoModel.from_pretrained(model_path, **model_kwargs).eval()
            
            if torch.cuda.is_available() and not CONFIG["enable_quantization"]:
                model = model.cuda()
        
        model_load_time = time.time() - model_start_time
        print(f"✅ {model_name} model loaded in {model_load_time:.1f}s")
        cleanup_gpu_memory()
        
        # Test each document with IMPROVED prompting
        for i, img_name in enumerate(verified_test_images, 1):
            expected_classification = verified_ground_truth[img_name]
            print(f"\n📄 Document {i}/{len(verified_test_images)}: {img_name} (expected: {expected_classification})")
            
            try:
                # Load image
                img_path = datasets_path / img_name
                image = Image.open(img_path).convert("RGB")
                
                inference_start = time.time()
                
                if model_name == "llama":
                    inputs = processor(text=classification_prompt, images=image, return_tensors="pt")
                    device = next(model.parameters()).device
                    if device.type != "cpu":
                        device_target = str(device).split(":")[0] if ":" in str(device) else str(device)
                        inputs = {k: v.to(device_target) if hasattr(v, "to") else v for k, v in inputs.items()}
                    
                    # RESEARCH-BASED: Deterministic generation
                    with torch.no_grad():
                        outputs = model.generate(
                            **inputs,
                            max_new_tokens=64,  # Short for classification
                            do_sample=False,    # Deterministic
                            temperature=None,   # Disable temperature
                            top_p=None,         # Disable top_p
                            top_k=None,         # Disable top_k
                            pad_token_id=processor.tokenizer.eos_token_id,
                            eos_token_id=processor.tokenizer.eos_token_id,
                            use_cache=True,
                        )
                    
                    raw_response = processor.decode(
                        outputs[0][inputs["input_ids"].shape[-1]:],
                        skip_special_tokens=True
                    )
                    
                    # Immediate cleanup of inference tensors
                    del inputs, outputs
                    
                elif model_name == "internvl":
                    transform = T.Compose([
                        T.Resize((448, 448), interpolation=InterpolationMode.BICUBIC),
                        T.ToTensor(),
                        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])
                    
                    pixel_values = transform(image).unsqueeze(0)
                    if torch.cuda.is_available():
                        pixel_values = pixel_values.cuda().to(torch.bfloat16).contiguous()
                    
                    raw_response = model.chat(
                        tokenizer=tokenizer,
                        pixel_values=pixel_values,
                        question=classification_prompt,
                        generation_config={"max_new_tokens": 64, "do_sample": False}
                    )
                    
                    if isinstance(raw_response, tuple):
                        raw_response = raw_response[0]
                    
                    # Immediate cleanup of inference tensors
                    del pixel_values
                
                inference_time = time.time() - inference_start
                
                # IMPROVED extraction: Handle JSON and text responses
                extracted_classification = "UNKNOWN"
                response_clean = raw_response.strip()
                
                # Try JSON extraction first
                if response_clean.startswith('{') and response_clean.endswith('}'):
                    try:
                        json_data = json.loads(response_clean)
                        if "document_type" in json_data:
                            extracted_classification = json_data["document_type"].upper()
                    except json.JSONDecodeError:
                        pass
                
                # Fallback to text extraction
                if extracted_classification == "UNKNOWN":
                    response_upper = response_clean.upper()
                    for doc_type in DOCUMENT_TYPES:
                        if doc_type in response_upper:
                            extracted_classification = doc_type
                            break
                
                # Calculate accuracy against human annotation
                is_correct = extracted_classification == expected_classification
                multi_doc_results[model_name]["total"] += 1
                if is_correct:
                    multi_doc_results[model_name]["correct"] += 1
                
                # Store results
                result = {
                    "image": img_name,
                    "predicted": extracted_classification,
                    "expected": expected_classification,
                    "correct": is_correct,
                    "inference_time": inference_time,
                    "raw_response": raw_response[:60] + "..." if len(raw_response) > 60 else raw_response
                }
                
                multi_doc_results[model_name]["classifications"].append(result)
                multi_doc_results[model_name]["times"].append(inference_time)
                
                # Show result
                status = "✅" if is_correct else "❌"
                print(f"   {status} {extracted_classification} ({inference_time:.1f}s)")
                if len(raw_response) < 100:
                    print(f"      Raw: {raw_response}")
                
                # Periodic memory cleanup every 3 images
                if i % 3 == 0:
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                
            except Exception as e:
                multi_doc_results[model_name]["errors"].append({
                    "image": img_name,
                    "expected": expected_classification,
                    "error": str(e)[:100]
                })
                multi_doc_results[model_name]["total"] += 1
                print(f"   ❌ ERROR: {str(e)[:60]}...")
        
        # AGGRESSIVE cleanup after model testing
        print(f"\n🧹 Cleaning up {model_name}...")
        del model
        if model_name == "llama":
            del processor
        elif model_name == "internvl":
            del tokenizer
        
        cleanup_gpu_memory()
        
        total_time = time.time() - model_start_time
        accuracy = multi_doc_results[model_name]["correct"] / multi_doc_results[model_name]["total"] * 100 if multi_doc_results[model_name]["total"] > 0 else 0
        
        print(f"\n📊 {model_name.upper()} SUMMARY:")
        print(f"   Accuracy: {accuracy:.1f}% ({multi_doc_results[model_name]['correct']}/{multi_doc_results[model_name]['total']})")
        print(f"   Total Time: {total_time:.1f}s")
        print(f"   Avg Time/Doc: {sum(multi_doc_results[model_name]['times'])/max(1,len(multi_doc_results[model_name]['times'])):.1f}s")
        
    except Exception as e:
        print(f"❌ {model_name.upper()} FAILED TO LOAD: {str(e)[:100]}...")
        
        # Emergency cleanup
        for var in ['model', 'processor', 'tokenizer', 'inputs', 'outputs', 'pixel_values']:
            if var in locals():
                del locals()[var]
        cleanup_gpu_memory()
        
        multi_doc_results[model_name]["model_error"] = str(e)

# Final Analysis with IMPROVED prompting results
print(f"\n{'=' * 80}")
print("🏆 IMPROVED PROMPTING ACCURACY ANALYSIS")
print(f"{'=' * 80}")

# Comparison table
comparison_data = []
comparison_data.append(["Image", "Expected", "Llama", "✓", "InternVL", "✓"])
comparison_data.append(["-" * 10, "-" * 10, "-" * 10, "-", "-" * 10, "-"])

llama_results = {r["image"]: r for r in multi_doc_results["llama"]["classifications"]}
internvl_results = {r["image"]: r for r in multi_doc_results["internvl"]["classifications"]}

for img_name in verified_test_images:
    expected = verified_ground_truth[img_name]
    llama_result = llama_results.get(img_name, {"predicted": "ERROR", "correct": False})
    internvl_result = internvl_results.get(img_name, {"predicted": "ERROR", "correct": False})
    
    comparison_data.append([
        img_name[:8],
        expected[:8],
        llama_result["predicted"][:8],
        "✅" if llama_result["correct"] else "❌",
        internvl_result["predicted"][:8],
        "✅" if internvl_result["correct"] else "❌"
    ])

for row in comparison_data:
    print(f"{row[0]:<10} {row[1]:<10} {row[2]:<10} {row[3]:<2} {row[4]:<10} {row[5]}")

# Final statistics with improvement comparison
print(f"\n📈 IMPROVED PROMPTING RESULTS:")
for model_name in ["llama", "internvl"]:
    if multi_doc_results[model_name]["total"] > 0:
        accuracy = multi_doc_results[model_name]["correct"] / multi_doc_results[model_name]["total"] * 100
        avg_time = sum(multi_doc_results[model_name]["times"]) / len(multi_doc_results[model_name]["times"])
        print(f"{model_name.upper()}: {accuracy:.1f}% accuracy, {avg_time:.2f}s/doc average")

# Final memory state
print(f"\n🧠 Final Memory State:")
cleanup_gpu_memory()

print(f"\n✅ Improved prompting classification completed!")
print(f"📋 Compare with previous results to see improvement")

🏛️ COMPREHENSIVE TAXPAYER DOCUMENT CLASSIFICATION TEST
🧪 Using IMPROVED research-based prompting techniques
📊 Testing 11 documents with HUMAN ANNOTATIONS:
   1. image14.png  → TAX_INVOICE
   2. image65.png  → TAX_INVOICE
   3. image71.png  → TAX_INVOICE
   4. image74.png  → TAX_INVOICE
   5. image205.png → FUEL_RECEIPT
   6. image23.png  → TAX_INVOICE
   7. image45.png  → TAX_INVOICE
   8. image1.png   → BANK_STATEMENT
   9. image203.png → BANK_STATEMENT
   10. image204.png → FUEL_RECEIPT
   11. image206.png → OTHER

🧪 Available classification prompts:
   - json_format: 322 chars
   - simple_format: 280 chars
   - ultra_simple: 23 chars

🔍 TESTING LLAMA WITH IMPROVED PROMPTING
🧹 Pre-cleanup for llama...
   GPU Memory: 0.03GB allocated, 0.03GB reserved
📝 Using SIMPLE FORMAT prompt (research-based)
Loading llama model from /home/jovyan/nfs_share/models/Llama-3.2-11B-Vision...
🔄 Loading Llama (will use ~6-8GB GPU memory)...
✅ Using 8-bit quantization


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ llama model loaded in 5.2s
   GPU Memory: 10.52GB allocated, 10.58GB reserved

📄 Document 1/11: image14.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.7s)

📄 Document 2/11: image65.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.8s)

📄 Document 3/11: image71.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.8s)

📄 Document 4/11: image74.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.7s)

📄 Document 5/11: image205.png (expected: FUEL_RECEIPT)
   ✅ FUEL_RECEIPT (5.7s)

📄 Document 6/11: image23.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.8s)

📄 Document 7/11: image45.png (expected: TAX_INVOICE)
   ❌ FUEL_RECEIPT (5.7s)

📄 Document 8/11: image1.png (expected: BANK_STATEMENT)
   ✅ BANK_STATEMENT (5.7s)

📄 Document 9/11: image203.png (expected: BANK_STATEMENT)
   ❌ FUEL_RECEIPT (5.6s)

📄 Document 10/11: image204.png (expected: FUEL_RECEIPT)
   ✅ FUEL_RECEIPT (5.8s)

📄 Document 11/11: image206.png (expected: OTHER)
   ❌ UNKNOWN (5.5s)

🧹 Cleaning up llama...
   GPU Memory: 0.03GB a

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


✅ 8-bit quantization enabled


...

   ✅ BANK_STATEMENT (1.5s)
      Raw: ```json
{
  "document_type": "BANK_STATEMENT"
}
```

📄 Document 9/11: image203.png (expected: BANK_STATEMENT)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


   ✅ BANK_STATEMENT (1.5s)
      Raw: ```json
{
  "document_type": "BANK_STATEMENT"
}
```

📄 Document 10/11: image204.png (expected: FUEL_RECEIPT)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


   ❌ TAX_INVOICE (1.5s)
      Raw: ```json
{
  "document_type": "TAX_INVOICE"
}
```

📄 Document 11/11: image206.png (expected: OTHER)
   ✅ OTHER (1.2s)
      Raw: ```json
{
  "document_type": "OTHER"
}
```

🧹 Cleaning up internvl...
   GPU Memory: 0.03GB allocated, 0.03GB reserved

📊 INTERNVL SUMMARY:
   Accuracy: 54.5% (6/11)
   Total Time: 21.1s
   Avg Time/Doc: 1.5s

🏆 IMPROVED PROMPTING ACCURACY ANALYSIS
Image      Expected   Llama      ✓  InternVL   ✓
---------- ---------- ---------- -  ---------- -
image14.   TAX_INVO   FUEL_REC   ❌  TAX_INVO   ✅
image65.   TAX_INVO   FUEL_REC   ❌  BUSINESS   ❌
image71.   TAX_INVO   FUEL_REC   ❌  TAX_INVO   ✅
image74.   TAX_INVO   FUEL_REC   ❌  MEAL_REC   ❌
image205   FUEL_REC   FUEL_REC   ✅  FUEL_REC   ✅
image23.   TAX_INVO   FUEL_REC   ❌  BUSINESS   ❌
image45.   TAX_INVO   FUEL_REC   ❌  BUSINESS   ❌
image1.p   BANK_STA   BANK_STA   ✅  BANK_STA   ✅
image203   BANK_STA   FUEL_REC   ❌  BANK_STA   ✅
image204   FUEL_REC   FUEL_REC   ✅  TAX_INVO   ❌


In [ ]:
# Information Extraction Comparison Test - Minimized Memory Footprint
print("🔬 INFORMATION EXTRACTION TEST: YAML Format with Memory Optimization")
print("=" * 70)

# Results storage for information extraction comparison  
extraction_results = {
    "llama": {"documents": [], "successful": 0, "total_time": 0},
    "internvl": {"documents": [], "successful": 0, "total_time": 0}
}

def cleanup_gpu_memory():
    """Minimize memory footprint as requested"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def analyze_yaml_extraction(response: str, img_name: str):
    """Analyze YAML extraction results with consistent format"""
    response_clean = response.strip()
    
    # Detect YAML format
    is_yaml = bool(re.search(r'(store_name:|date:|total:)', response_clean, re.IGNORECASE))
    
    # Extract data from YAML or text
    store_match = re.search(r'store_name:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
    date_match = re.search(r'date:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)  
    total_match = re.search(r'total:\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
    
    # Fallback detection for non-YAML responses
    if not store_match:
        store_match = re.search(r'(spotlight|store)', response_clean, re.IGNORECASE)
    if not date_match:
        date_match = re.search(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', response_clean)
    if not total_match:
        total_match = re.search(r'(\$\d+\.\d{2}|\$\d+)', response_clean)
    
    has_store = bool(store_match)
    has_date = bool(date_match)
    has_total = bool(total_match)
    
    extraction_score = sum([has_store, has_date, has_total])
    
    return {
        "img_name": img_name,
        "response": response_clean,
        "is_yaml": is_yaml,
        "has_store": has_store,
        "has_date": has_date,
        "has_total": has_total,
        "extraction_score": extraction_score,
        "successful": extraction_score >= 2  # At least 2/3 fields
    }

# Test each model for information extraction
test_models = ["llama", "internvl"]

for model_name in test_models:
    print(f"\n{'=' * 60}")
    print(f"🔬 TESTING {model_name.upper()} INFORMATION EXTRACTION")
    print(f"{'=' * 60}")
    
    cleanup_gpu_memory()
    model_start_time = time.time()
    
    try:
        model_path = CONFIG["model_paths"][model_name]
        print(f"Loading {model_name} model with 8-bit quantization...")
        
        if model_name == "llama":
            from transformers import AutoProcessor, MllamaForConditionalGeneration
            from transformers import BitsAndBytesConfig
            
            processor = AutoProcessor.from_pretrained(
                model_path, trust_remote_code=True, local_files_only=True
            )
            
            quantization_config = BitsAndBytesConfig(
                load_in_8bit=True,
                llm_int8_enable_fp32_cpu_offload=True,
                llm_int8_skip_modules=["vision_tower", "multi_modal_projector"],
            )
            
            model = MllamaForConditionalGeneration.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                torch_dtype=torch.float16,
                local_files_only=True
            ).eval()
            
        elif model_name == "internvl":
            from transformers import AutoModel, AutoTokenizer
            import torchvision.transforms as T
            from torchvision.transforms.functional import InterpolationMode
            
            tokenizer = AutoTokenizer.from_pretrained(
                model_path, trust_remote_code=True, local_files_only=True
            )
            
            model = AutoModel.from_pretrained(
                model_path,
                load_in_8bit=True,
                trust_remote_code=True,
                torch_dtype=torch.bfloat16,
                local_files_only=True
            ).eval()
        
        model_load_time = time.time() - model_start_time
        print(f"✅ Model loaded in {model_load_time:.1f}s")
        
        # Test image14.png first to validate working parameters
        print(f"\n🔍 Testing image14.png first...")
        img_path = datasets_path / "image14.png"
        image = Image.open(img_path).convert("RGB")
        
        inference_start = time.time()
        
        if model_name == "llama":
            inputs = processor(text=CONFIG["extraction_prompt"], images=image, return_tensors="pt")
            device = next(model.parameters()).device
            if device.type != "cpu":
                device_target = str(device).split(":")[0]
                inputs = {k: v.to(device_target) if hasattr(v, "to") else v for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=CONFIG["max_new_tokens"],
                    do_sample=False,
                    pad_token_id=processor.tokenizer.eos_token_id,
                    eos_token_id=processor.tokenizer.eos_token_id,
                    use_cache=True,
                )
            
            raw_response = processor.decode(
                outputs[0][inputs["input_ids"].shape[-1]:],
                skip_special_tokens=True
            )
            del inputs, outputs
            
        elif model_name == "internvl":
            transform = T.Compose([
                T.Resize((448, 448), interpolation=InterpolationMode.BICUBIC),
                T.ToTensor(),
                T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ])
            
            pixel_values = transform(image).unsqueeze(0)
            if torch.cuda.is_available():
                pixel_values = pixel_values.cuda().to(torch.bfloat16).contiguous()
            
            raw_response = model.chat(
                tokenizer=tokenizer,
                pixel_values=pixel_values,
                question=CONFIG["extraction_prompt"],
                generation_config={"max_new_tokens": CONFIG["max_new_tokens"], "do_sample": False}
            )
            
            if isinstance(raw_response, tuple):
                raw_response = raw_response[0]
            
            del pixel_values
        
        inference_time = time.time() - inference_start
        cleaned_response = repetition_controller.clean_response(raw_response)
        validation_analysis = analyze_yaml_extraction(cleaned_response, "image14.png")
        
        print(f"   Validation result: {validation_analysis['extraction_score']}/3 fields extracted")
        print(f"   YAML format: {'✅' if validation_analysis['is_yaml'] else '❌'}")
        print(f"   Inference time: {inference_time:.1f}s")
        
        if validation_analysis["successful"]:
            print(f"✅ Validation passed - proceeding with all {len(verified_extraction_images)} documents")
            
            total_inference_time = 0
            
            for i, (img_name, doc_type) in enumerate(verified_extraction_images, 1):
                try:
                    img_path = datasets_path / img_name
                    image = Image.open(img_path).convert("RGB")
                    
                    inference_start = time.time()
                    
                    if model_name == "llama":
                        inputs = processor(text=CONFIG["extraction_prompt"], images=image, return_tensors="pt")
                        device = next(model.parameters()).device
                        if device.type != "cpu":
                            device_target = str(device).split(":")[0]
                            inputs = {k: v.to(device_target) if hasattr(v, "to") else v for k, v in inputs.items()}
                        
                        with torch.no_grad():
                            outputs = model.generate(
                                **inputs,
                                max_new_tokens=CONFIG["max_new_tokens"],
                                do_sample=False,
                                pad_token_id=processor.tokenizer.eos_token_id,
                                eos_token_id=processor.tokenizer.eos_token_id,
                                use_cache=True,
                            )
                        
                        raw_response = processor.decode(
                            outputs[0][inputs["input_ids"].shape[-1]:],
                            skip_special_tokens=True
                        )
                        del inputs, outputs
                        
                    elif model_name == "internvl":
                        transform = T.Compose([
                            T.Resize((448, 448), interpolation=InterpolationMode.BICUBIC),
                            T.ToTensor(),
                            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                        ])
                        
                        pixel_values = transform(image).unsqueeze(0)
                        if torch.cuda.is_available():
                            pixel_values = pixel_values.cuda().to(torch.bfloat16).contiguous()
                        
                        raw_response = model.chat(
                            tokenizer=tokenizer,
                            pixel_values=pixel_values,
                            question=CONFIG["extraction_prompt"],
                            generation_config={"max_new_tokens": CONFIG["max_new_tokens"], "do_sample": False}
                        )
                        
                        if isinstance(raw_response, tuple):
                            raw_response = raw_response[0]
                        
                        del pixel_values
                    
                    inference_time = time.time() - inference_start
                    total_inference_time += inference_time
                    
                    cleaned_response = repetition_controller.clean_response(raw_response)
                    analysis = analyze_yaml_extraction(cleaned_response, img_name)
                    analysis["inference_time"] = inference_time
                    analysis["doc_type"] = doc_type
                    
                    extraction_results[model_name]["documents"].append(analysis)
                    
                    if analysis["successful"]:
                        extraction_results[model_name]["successful"] += 1
                    
                    # Consistent output format as requested
                    status = "✅" if analysis["successful"] else "❌"
                    yaml_status = "Y" if analysis["is_yaml"] else "T"
                    print(f"   {i:2d}. {img_name:<12} {status} {inference_time:.1f}s | {yaml_status} | {analysis['extraction_score']}/3")
                    
                    # Immediate tensor cleanup - minimizing memory footprint
                    del image
                    
                    # Periodic GPU cleanup every 3 images
                    if i % 3 == 0:
                        gc.collect()
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                    
                except Exception as e:
                    print(f"   {i:2d}. {img_name:<12} ❌ Error: {str(e)[:30]}...")
            
            extraction_results[model_name]["total_time"] = total_inference_time
            extraction_results[model_name]["avg_time"] = total_inference_time / len(verified_extraction_images)
            
        else:
            print(f"❌ Validation failed - skipping full test")
        
        # Cleanup model to minimize memory footprint
        del model
        if model_name == "llama":
            del processor
        elif model_name == "internvl":
            del tokenizer
        cleanup_gpu_memory()
        
    except Exception as e:
        print(f"❌ {model_name.upper()} FAILED TO LOAD: {str(e)[:100]}...")

# FINAL COMPARISON: Information Extraction Performance
print(f"\n{'=' * 80}")
print("🏆 FINAL RECOMMENDATION: BEST MODEL FOR INFORMATION EXTRACTION")
print(f"{'=' * 80}")

# Compare both models' performance
llama_success = 0
llama_total = 0
llama_avg_time = 0
internvl_success = 0
internvl_total = 0
internvl_avg_time = 0

if extraction_results["llama"]["documents"]:
    llama_total = len(extraction_results["llama"]["documents"])
    llama_success = extraction_results["llama"]["successful"]
    llama_avg_time = extraction_results["llama"]["avg_time"]

if extraction_results["internvl"]["documents"]:
    internvl_total = len(extraction_results["internvl"]["documents"])
    internvl_success = extraction_results["internvl"]["successful"]
    internvl_avg_time = extraction_results["internvl"]["avg_time"]

print(f"📊 INFORMATION EXTRACTION COMPARISON:")
print(f"{'Model':<12} {'Success Rate':<15} {'Avg Time':<12} {'Best For'}")
print("-" * 60)

if llama_total > 0:
    llama_rate = llama_success / llama_total * 100
    print(f"{'LLAMA':<12} {llama_rate:.1f}% ({llama_success}/{llama_total}){'':<5} {llama_avg_time:.1f}s{'':<7} Large context")

if internvl_total > 0:
    internvl_rate = internvl_success / internvl_total * 100
    print(f"{'INTERNVL':<12} {internvl_rate:.1f}% ({internvl_success}/{internvl_total}){'':<5} {internvl_avg_time:.1f}s{'':<7} Production speed")

# Make recommendation
if internvl_total > 0 and llama_total > 0:
    internvl_rate = internvl_success / internvl_total * 100
    llama_rate = llama_success / llama_total * 100
    
    if internvl_rate > llama_rate:
        recommended = "INTERNVL"
        reason = f"Higher success rate ({internvl_rate:.1f}% vs {llama_rate:.1f}%) and faster inference"
    elif llama_rate > internvl_rate:
        recommended = "LLAMA"
        reason = f"Higher success rate ({llama_rate:.1f}% vs {internvl_rate:.1f}%)"
    else:
        recommended = "INTERNVL"
        reason = f"Equal success rate but {internvl_avg_time/llama_avg_time:.1f}x faster inference"
    
    print(f"\n🥇 RECOMMENDED FOR INFORMATION EXTRACTION: {recommended}")
    print(f"   Reason: {reason}")
    print(f"   Use case: Business document processing (receipts, invoices, statements)")
elif internvl_total > 0:
    print(f"\n🥇 RECOMMENDED: INTERNVL (only model tested successfully)")
elif llama_total > 0:
    print(f"\n🥇 RECOMMENDED: LLAMA (only model tested successfully)")
else:
    print(f"\n⚠️ No successful tests - investigate model loading issues")

print(f"\n✅ COMPLETE: Information extraction performance comparison finished!")
print(f"📋 This answers the user's question about best model for their information extraction job")